In [1]:
import cv2 as cv
import os
import numpy as np

In [2]:
def preprocess_image_level(img_path):
    
    img = cv.imread(img_path)
    
    resized_img = cv.resize(img, (640, 640))

    denoised_img1 = cv.bilateralFilter(resized_img, 15, 21, 21)

    grey = cv.cvtColor(denoised_img1, cv.COLOR_BGR2GRAY)

    adaptive_thresh = cv.adaptiveThreshold(
        grey, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV, 11, 3
    )

    kernel = np.ones((3, 3), np.uint8)
    dilate = cv.dilate(adaptive_thresh, kernel, iterations=1)

    normalize = cv.normalize(
        dilate,
        None,
        alpha=10,
        beta=300,
        norm_type=cv.NORM_MINMAX,
        dtype=cv.WARP_INVERSE_MAP,
    )

    mask = cv.bitwise_not(normalize)
    return mask

In [3]:
img_path='/home/uwu/Desktop/object_detection/preview/yolo/1f9e8765-44db-4303-b12a-d6b30c9cc5f9.png'

In [4]:


mask = preprocess_image(img_path)

cv.imshow('test',mask)
cv.waitKey(0)
cv.destroyAllWindows()